<a href="https://colab.research.google.com/github/rod31/succession-nlp/blob/main/sarcasm_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

csv_path = '/content/drive/My Drive/Data Science Projects/RoyGPT/roman_roy_sarcasm_labels.csv'
df = pd.read_csv(csv_path)

# Quick sanity check
print(df.shape)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(2466, 2)


,text,Is_Sarcastic
0,"Hey hey hey, motherfuckers!",NaN
1,My guy? Are you saging?,NaN
2,"Right, bad juju. Maybe you should make a move,...",NaN
3,I think just fuck off thanks. How’s it going?,NaN
4,Of course I’m okay. Obviously I’m okay. Why do...,NaN


Attempt #1:
Is_Sarcastic field is not labeled so I try to label it with Zero Shot Classification.

Zero-shot classification is a machine learning technique where a model can classify data into categories it has never seen before.

In [6]:
from transformers import pipeline
# Load a zero-shot classifier
zs = pipeline(
  "zero-shot-classification",
  model="facebook/bart-large-mnli"
)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [6]:
# Candidate labels
candidate_labels = ["Sarcastic", "Not Sarcastic"]

# 3. Apply to your entire dataframe (be careful with very large batches!):
batch_size = 32
zs_preds = []
for i in range(0, len(df), batch_size):
    batch = df['text'].iloc[i:i+batch_size].tolist()
    results = zs(batch, candidate_labels)
    # grab top label for each
    zs_preds.extend([r['labels'][0] for r in results])

# 4. Store back into the DataFrame
df['ZS_Label'] = zs_preds

# 5. Quick distribution
print(df['ZS_Label'].value_counts())
df.head(20)

ZS_Label
Sarcastic        1249
Not Sarcastic    1217
Name: count, dtype: int64


,text,Is_Sarcastic,ZS_Label
0,"Hey hey hey, motherfuckers!",NaN,Sarcastic
1,My guy? Are you saging?,NaN,Sarcastic
2,"Right, bad juju. Maybe you should make a move,...",NaN,Sarcastic
3,I think just fuck off thanks. How’s it going?,NaN,Sarcastic
4,Of course I’m okay. Obviously I’m okay. Why do...,NaN,Sarcastic
5,What that’s ‘commercially sensitive’ – I’m sti...,NaN,Not Sarcastic
6,One-twenty-five! Fuck!?,NaN,Sarcastic
7,"Hey, Global Tom! How you shaking? You still fu...",NaN,Sarcastic
8,Sis. The pols still boring the living shit out...,NaN,Sarcastic
9,Ha! Look at you.* Walking around like you’re a...,NaN,Sarcastic


Attempt #2: Off-the-Shelf Sarcasm Detector

There are HuggingFace models fine-tuned directly for sarcasm. Here’s an example using mohammadhf/roberta-base-sarcasm-detector:

Helinivan (English Sarcasm) didn't work

T5 (Sarcasm Twitter) didn't work

Let's compare Zero Shot Sarcasm detection against my manual labeling of Season 4

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 1. Filter down to only the rows you manually labeled
#    (i.e. Is_Sarcastic is not NaN)
labeled_df = df[df['Is_Sarcastic'].notna()].copy()

# 2. Convert the zero‐shot labels into 0/1
labeled_df['ZS_bin'] = (labeled_df['ZS_Label'] == 'Sarcastic').astype(int)

# 3. True labels (manually labeled)
y_true = labeled_df['Is_Sarcastic'].astype(int)

# 4. Predicted labels
y_pred = labeled_df['ZS_bin']

# 5. Compute metrics
print("Accuracy:", accuracy_score(y_true, y_pred))
print("\nClassification Report:")
print(classification_report(y_true, y_pred,
      target_names=['Not Sarcasm','Sarcasm']))
print("\nConfusion Matrix:")
cm = confusion_matrix(y_true, y_pred)
print(cm)

KeyError: 'ZS_Label'